## Observations and Insights 

In [1]:
%matplotlib widget

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset and display the data table for preview
combined_mousestudy_df = pd.merge(mouse_metadata, study_results,
                                 how='left', on='Mouse ID')

# sorting by mouse id  
combined_mousestudy_df.sort_values(["Mouse ID", "Timepoint"], inplace = True) 
      
# dropping ALL duplicte values 
combined_data_clean = combined_mousestudy_df.drop_duplicates(subset =["Mouse ID", "Timepoint"], 
                keep = False, inplace =True) 
 
    
    
    
    
    
    
#Display data
# combined_mousestudy_df["Tumor Volume (mm3)"] = combined_mousestudy_df["Tumor Volume (mm3)"].map('{:,.2f}'.format)
combined_data_clean


In [ ]:
#len(combined_data_clean['Mouse ID'].unique())

## Summary Statistics

In [ ]:
# Summary statistics table 
# mean, median, variance, standard deviation, and SEM (standard error of the mean) of the tumor volume for each regimen

tumor_volume_avg = combined_mousestudy_df["Tumor Volume (mm3)"].mean()
tumor_volume_median = combined_mousestudy_df["Tumor Volume (mm3)"].median()
tumor_volume_variance = combined_mousestudy_df["Tumor Volume (mm3)"].var()
tumor_volume_standev = combined_mousestudy_df["Tumor Volume (mm3)"].std()
tumor_volume_sem = combined_mousestudy_df["Tumor Volume (mm3)"].sem()

Summary_tumor_volume = pd.DataFrame({'Avg Tumor Volume': [tumor_volume_avg], 'Mediam Tumor Volume': [tumor_volume_median], 
                                     'Variance': [tumor_volume_variance], 'Standard Deviation': [tumor_volume_standev], 
                                     'Standard Error of Mean': [tumor_volume_sem]})
Summary_tumor_volume

## Bar and Pie Charts

In [ ]:
# Generate a bar plot showing the total number of mice for each treatment throughout the course of the study using pandas. 



In [ ]:
# Generate a bar plot showing the total number of mice for each treatment throughout the course of the study using pyplot.



In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pandas



In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot



## Quartiles, Outliers and Boxplots

In [ ]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original dataframe to get the tumor volume at the last timepoint


In [ ]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds
    

In [ ]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest


## Line and Scatter Plots

In [ ]:
# Generate a line plot of time point versus tumor volume for a mouse treated with Capomulin


In [ ]:
# Generate a scatter plot of mouse weight versus average tumor volume for the Capomulin regimen


## Correlation and Regression

In [ ]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
